# Resonance model train+test

In [9]:
import os
import argparse
from pydub import AudioSegment
#trim audio and save
import numpy
import math
import scipy.io.wavfile

#gneral utils
formats_to_convert = ['.m4a']
def trans_to_wav(dirpath):
    for (dirpath, dirnames, filenames) in os.walk(dirpath):
        for filename in filenames:
            if filename.endswith(tuple(formats_to_convert)):

                filepath = dirpath + '/' + filename
                (path, file_extension) = os.path.splitext(filepath)
                file_extension_final = file_extension.replace('.', '')
                try:
                    track = AudioSegment.from_file(filepath,
                            file_extension_final)
                    wav_filename = filename.replace(file_extension_final, 'wav')
                    wav_path = dirpath + '/' + wav_filename
                    print('CONVERTING: ' + str(filepath))
                    file_handle = track.export(wav_path, format='wav')
                    os.remove(filepath)
                except:
                    print("ERROR CONVERTING " + str(filepath))

def cut_audio_save(filepath,start,end,new_name):
    sound = AudioSegment.from_file(filepath)
    demo = sound[start:end]#in ms
    demo.export(new_name+".wav", format="wav")
    
    
dirpath1=r"C:\Users\gaoyu\OneDrive\Documents\Sound recordings"
dirpath2=r"C:\Users\gaoyu\Desktop\data_resonance"

In [10]:
#feature utils
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc

 
def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""
    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j==rows-1:#pay attenstion here original text i-j rows-1
                second = rows-1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas
 
def extract_features(audio,rate):
    """extract 20 dim mfcc features from an audio, performs CMS and combines
    delta to make it 40 dim feature vector"""   

    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True)
    mfcc_feat = preprocessing.scale(mfcc_feat)
    #delta = calculate_delta(mfcc_feat)
    #combined = np.hstack((mfcc_feat,delta))
    combined = np.vstack((mfcc_feat,mfcc_feat))
    return combined

In [11]:
#model training
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GMM
import warnings
#file setup
#0.660694-4.068803-6.604524-10 (0.2 persample)
#no resonance-forward-backward-nasal
file_to_cut=r"C:\Users\gaoyu\Desktop\music try\data_resonance\all_types_F_chain.wav"
folder_to_save="resonance_set\\" 
#cut_audio_save(filepath,start,end,new_name)
def prep_data(filepath,dirpath):
    sound = AudioSegment.from_file(filepath)
    length=len(sound)#9929ms 
    duration=200
    intervals=length//duration
    L=[]
    
    for i in range(intervals):
        start=i*duration
        if i==intervals-1:
            end=length
        else:
            end=(i+1)*duration
        demo = sound[start:end]#in ms
        #name set
        if end<661:
            new_name="no"
        elif end<4061:
            new_name="forward"
        elif end<6604:
            new_name="backward"
        else:
            new_name="nasal"
        file_name=new_name+"-"+str(i)+".wav"
        L.append(file_name+"\n")
        demo.export(dirpath+file_name, format="wav")
    
    file1 = open("development_set_enroll.txt","w") 
    file1.writelines(L) 
    file1.close()
    
    
prep_data(file_to_cut,folder_to_save)    


9929
49
48


In [12]:
#GMM part
#model training
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GMM
import warnings
warnings.filterwarnings("ignore")
 
#path to training data
source   = "resonance_set\\"  
 
#path where training speakers will be saved
dest = "resonance_models\\"
train_file = r"C:\Users\gaoyu\Desktop\music try\data_resonance\development_set_enroll.txt"
file_paths = open(train_file,'r')
 
#path of file to train="resonancetype-number"    
count = 1
# Extracting features for each resonance (5 files per resonance styles)
features = np.asarray(())
for path in file_paths:
    path = path.strip() 
    # read the audio
    sr,audio = read(source + path)
    vector = extract_features(audio,sr)
    if features.size == 0:
        features = vector
    else:
        try:
            features = np.vstack((features, vector))
        except:
            pass
    # when features of 5 files of speaker are concatenated, then do model training
    if count == 3:
        gmm = GMM(n_components = 3, n_iter = 200, covariance_type='diag',n_init = 3)
        gmm.fit(features)
 
        # dumping the trained gaussian model
        picklefile = path.split("-")[0]+".gmm"
        cPickle.dump(gmm,open(dest + picklefile,'wb'))
        print ('+ modeling completed for resonance type:',picklefile," with data point = ",features.shape)
        features = np.asarray(())
        #count = 0
    elif count == 20:
        gmm = GMM(n_components = 20-3, n_iter = 200, covariance_type='diag',n_init = 3)
        gmm.fit(features)
 
        # dumping the trained gaussian model
        picklefile = path.split("-")[0]+".gmm"
        cPickle.dump(gmm,open(dest + picklefile,'wb'))
        print ('+ modeling completed for resonance type:',picklefile," with data point = ",features.shape)
        features = np.asarray(())
        #count = 0
    elif count == 33:
        gmm = GMM(n_components = 33-20, n_iter = 200, covariance_type='diag',n_init = 3)
        gmm.fit(features)
 
        # dumping the trained gaussian model
        picklefile = path.split("-")[0]+".gmm"
        cPickle.dump(gmm,open(dest + picklefile,'wb'))
        print ('+ modeling completed for resonance type:',picklefile," with data point = ",features.shape)
        features = np.asarray(())
        #count = 0
    elif count == 49:
        
        gmm = GMM(n_components = 15, n_iter = 200, covariance_type='diag',n_init = 3)
        gmm.fit(features)
 
        # dumping the trained gaussian model
        picklefile = path.split("-")[0]+".gmm"
        cPickle.dump(gmm,open(dest + picklefile,'wb'))
        print ('+ modeling completed for resonance type:',picklefile," with data point = ",features.shape)
        features = np.asarray(())
        #count = 0
    count = count + 1
   
  
        
    


+ modeling completed for resonance type: no.gmm  with data point =  (234, 20)


+ modeling completed for resonance type: forward.gmm  with data point =  (1326, 20)


+ modeling completed for resonance type: backward.gmm  with data point =  (1014, 20)
+ modeling completed for resonance type: nasal.gmm  with data point =  (1300, 20)


In [16]:
#test models:
import os
import numpy as np
from scipy.io.wavfile import read
import warnings
warnings.filterwarnings("ignore")
import time
 
#path to training data
source   = "development_set\\"
modelpath = "resonance_models\\"

test_file = r"C:\Users\gaoyu\Desktop\music try\data_resonance\development_set_test.txt"
file_paths = open(test_file,'r')
gmm_files = [os.path.join(modelpath,fname) for fname in
              os.listdir(modelpath) if fname.endswith('.gmm')]
 
#Load the Gaussian gender Models
models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]
resonances   = [fname.split("\\")[-1].split(".gmm")[0] for fname
              in gmm_files] 
# Read the test directory and get the list of test audio files
for path in file_paths:   
    path = path.strip()
    print(path)
    sr,audio = read(path)
    vector   = extract_features(audio,sr)
    #vector=vector.reshape((1, -1))
    log_likelihood = np.zeros(len(models)) 
 
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()

    winner = np.argmax(log_likelihood)
    print(winner)
    print ("\tdetected as - ", resonances[winner])
    time.sleep(1.0)

f_male_baseline.wav
[-7433.01897468 -7338.11019522 -7377.10246851 -7521.58747878]
1
	detected as -  forward


backward_F4_a_male.wav
[-25154.26943587 -25562.91923749 -25691.30938192 -25513.04952436]
0
	detected as -  backward


backward_lowered_larynx_male_a_F.wav
[-27627.62109675 -29646.34552096 -28976.62233508 -28545.1753033 ]
0
	detected as -  backward


demo_female_forward.wav
[-16947.23970618 -17283.41270666 -17407.68523388 -17404.8676683 ]
0
	detected as -  backward


demo_female_backward.wav
[-30478.44812965 -31917.38921853 -32073.21002835 -30651.54357625]
0
	detected as -  backward


demo_female_trans_for.wav
[-7760.63953085 -7868.93703151 -7850.75846904 -7849.44534685]
0
	detected as -  backward


demo_female_trans_back.wav
[-14400.1742067  -15484.59127426 -15533.27017631 -14886.5683451 ]
0
	detected as -  backward


# Vibrato 

In [ ]:
import scipy.io.wavfile as wav
import numpy as np
import os
import csv
from scipy.fftpack import fft
import pandas
import os
NFFT = 512
F_low = 5
F_high = 8

'''
功能：提取颤音特征
输入：time-pitch序列（间隔5ms）
输出：颤音特征：Likelihood,Rate,Extent
'''
def VibratoFeature(time_pitch):
    NFFT = 512
    pitch_hop = 0.005
    freq = np.arange(0, 1 / (2 * pitch_hop), (1 / (pitch_hop * NFFT)))

    time = time_pitch[:,0]
    pitch = time_pitch[:,1]
    vib_features = np.array([])

    cur_time = 0
    time_length = 0.5 #分析长度为0.5s
    time_hop = 0.25 #滑动为0.25秒
    while cur_time<time[-1]:
        time_stamp = np.array([cur_time,cur_time+time_length])
        
        features = []
        n1 = np.argmin(np.abs(np.subtract(time, time_stamp[0])))
        n2 = np.argmin(np.abs(np.subtract(time, time_stamp[1])))
        pitch_snippet = pitch[n1:n2+1]
        time_snippet = time[n1:n2+1]

        pitch_snippet = pitch_snippet - np.mean(pitch_snippet)

        ### Likelihood
        X = np.abs(fft(pitch_snippet * np.hamming(len(pitch_snippet)), NFFT))
        Xhalf_norm = X[0:int(NFFT / 2)] / sum(X[0:int(NFFT / 2)])
        vib_region = Xhalf_norm[(freq >= F_low) & (freq <= F_high)]
        vib_power = sum(vib_region)
        vib_sum_power = sum(Xhalf_norm)
        vib_power_ratio = vib_power/vib_sum_power
        features.append(vib_power_ratio)

        ### Rate
        zero_crossings = np.where(np.diff(np.sign(pitch_snippet)))[0]
        time_zerocrossings = time_snippet[zero_crossings[0::2]]
        if len(time_zerocrossings) <=1: #不够一个周期==time_zerocrossings不到两个
            if vib_features.size == 0:
                vib_features = np.array([0.,0.,0.])
            else:
                vib_features = np.vstack((vib_features, np.array([0.,0.,0.])))
            cur_time+=time_hop
            continue
        rate = 1.0/((1.0/(len(time_zerocrossings)-1))*sum(np.abs(time_zerocrossings[0:-1]-time_zerocrossings[1:])))
        features.append(rate)

        ### Extent
        pitch_max = []
        for index in range(len(zero_crossings)-1):
            pitch_max.append(max(np.abs(pitch_snippet[zero_crossings[index]:zero_crossings[index+1]])))
        pitch_max = np.array(pitch_max)
        extent =  (0.5/(len(pitch_max)-1))*sum(np.abs(pitch_max[0:-1]+pitch_max[1:]))
        features.append(extent)

        if vib_features.size == 0:
            vib_features = np.array(features)
        else:
            vib_features = np.vstack((vib_features,np.array(features)))
        # print(features)
        cur_time+=time_hop
    return vib_features

# Vibroato test

In [ ]:
##############################按照文件夹遍历##############################
# base_dir = "G:/data/"
audio_name = "27.wav"
audio_path = "temp/"+audio_name
pitch_path = "temp/"+audio_name[:-4]+"_pitch.csv"
pitch_path2 = "temp/"+audio_name[:-4]+"_pitch_5ms.csv"
note_path = "temp/"+audio_name[:-4]+"_note.csv"
#########################################################################

(fs, ori_sig) = wav.read(audio_path)
window = NFFT / (fs * 1.0)
hop = window / 2.0
#############################如果以前没有提取过pitch##################################
if not os.path.exists(pitch_path2):
    time_hop = (1000/44100)*256
    command = "./AudioBaseTestPitch0625 "+audio_path+" "+pitch_path+" "+note_path+" "+pitch_path2
    print(command)
    os.system(command)
    #########################################################################
    #padding补0
    data = pandas.read_csv(pitch_path2).values
    data = data[np.lexsort(data[:,::-1].T)]
    last_time = (len(ori_sig)/fs)*1000

    #旧的pitch path 2可以删掉
    command2 = "rm "+pitch_path2 
    os.system(command2)

    #以5ms为边界，重写pitch path
    time_hop = 5
    f = open(pitch_path2,'w')
    cur_time = 0 #curtime为时间戳
    j = 0 #j为遍历data的序号
    while cur_time < last_time:
        if j>=data.shape[0] or abs(cur_time-data[j][0])>=(time_hop/2): #以time_hop为边界
            f.write(str(round(cur_time/1000.0,3))+" 0"+'\n')
        else:
            f.write(str(round(data[j][0]/1000.0,3))+" "+str(data[j][1])+'\n')
            j+=1
        cur_time+=time_hop
    f.close()
#############################读取pitch_time##################################
f = open(pitch_path2, 'r')
obj = csv.reader(f, delimiter=' ')
cols = []
for row in obj:
    if len(row)<2: break
    if np.size(cols) == 0:
        cols = [float(elem) for elem in row]
    else:
        cols = np.vstack((cols, [float(elem) for elem in row]))
f.close()
time_pitch = cols
############################提取颤音特征#################################
vib_features = VibratoFeature(time_pitch)

for i in vib_features:
    print(i)

# Combine res+vib==>technique_vector

In [17]:
#To get technique vector for each file:

def Resonance_feature(path,modelpath):
    gmm_files = [os.path.join(modelpath,fname) for fname in
    os.listdir(modelpath) if fname.endswith('.gmm')]
    #Load the Gaussian gender Models
    models    = [cPickle.load(open(fname,'rb')) for fname in gmm_files]

    res_features = np.array([])
    features = []
    res_features=np.asarray(features)
    #calculate log_likelihood(4,) for frame of 0.5ms and concanentate (4,frame_num)
    sr,audio = read(path)
    #TODO:cut the audio to sections of 0.5ms and loop to calculate each vector
    #loop starts
    vector = extract_features(audio,sr)
    #vector=vector.reshape((1, -1))
    log_likelihood = np.zeros(len(models)) 
    for i in range(len(models)):
        gmm    = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
    features.extend(log_likelihood)
    #loop ends
    res_feature=np.asarray(features)
    return res_feature

#######################################################################
#0.get model path
modelpath = "resonance_models\\"
#1.get resonance feature
res_feature=Resonance_future(path,modelpath)
#2.get time-pitch series
time_pitch=get_time_pitch_series(path)
#get vib_feature
vib_features = VibratoFeature(time_pitch)
#stack with vibrato features(3,) for frame of 0.5ms and concanenate (3,frame_num) 
########################################################################
#stack to get (7,frame_num)
technique_vector=np.vstack((vib_features,res_features))
#pass technique_vector to model

SyntaxError: unexpected EOF while parsing (<ipython-input-17-15fcf27b3fb0>, line 21)